# In-Depth: Decision Trees and Random Forests

> This notebook was adapted from [PythonDataScienceHandbook's GitHub](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.08-Random-Forests.ipynb). Have a look to the repository: it contains great examples for many models.


We'll take a look at another powerful algorithm—a non-parametric algorithm called *random forests*.
Random forests are an example of an *ensemble* method, meaning that it relies on aggregating the results of an ensemble of simpler estimators.
The somewhat surprising result with such ensemble methods is that the sum can be greater than the parts: that is, a majority vote among a number of estimators can end up being better than any of the individual estimators doing the voting!
We will see examples of this in the following sections.
We begin with the standard imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("seaborn-whitegrid")
plt.rcParams["figure.figsize"] = (16, 9)

## Motivating Random Forests: Decision Trees

Random forests are an example of an *ensemble learner* built on decision trees.
For this reason we'll start by discussing decision trees themselves.

Decision trees are extremely intuitive ways to classify or label objects: you simply ask a series of questions designed to zero-in on the classification.
For example, if you wanted to build a decision tree to classify an animal you come across while on a hike, you might construct the one shown here:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.08-decision-tree.png?raw=1)

The binary splitting makes this extremely efficient: in a well-constructed tree, each question will cut the number of options by approximately half, very quickly narrowing the options even among a large number of classes.
The trick, of course, comes in deciding which questions to ask at each step.
In machine learning implementations of decision trees, the questions generally take the form of axis-aligned splits in the data: that is, each node in the tree splits the data into two groups using a cutoff value within one of the features.
Let's now look at an example of this.

### Creating a decision tree

Consider the following two-dimensional data, which has one of four class labels:

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=300, centers=4, random_state=0, cluster_std=1.0)
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap="rainbow");

A simple decision tree built on this data will iteratively split the data along one or the other axis according to some quantitative criterion, and at each level assign the label of the new region according to a majority vote of points within it.
This figure presents a visualization of the first four levels of a decision tree classifier for this data:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.08-decision-tree-levels.png?raw=1)

Notice that after the first split, every point in the upper branch remains unchanged, so there is no need to further subdivide this branch.
Except for nodes that contain all of one color, at each level *every* region is again split along one of the two features.

This process of fitting a decision tree to our data can be done in Scikit-Learn with the ``DecisionTreeClassifier`` estimator:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier().fit(X, y)

Let's write a quick utility function to help us visualize the output of the classifier:

In [ ]:
def visualize_classifier(model, X, y, ax=None, cmap="rainbow"):
    ax = ax or plt.gca()

    # Plot the training points
    ax.scatter(
        X[:, 0], X[:, 1], c=y, s=30, cmap=cmap, clim=(y.min(), y.max()), zorder=3
    )
    ax.axis("tight")
    ax.axis("off")
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # fit the estimator
    model.fit(X, y)
    xx, yy = np.meshgrid(np.linspace(*xlim, num=200), np.linspace(*ylim, num=200))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Create a color plot with the results
    n_classes = len(np.unique(y))
    ax.contourf(
        xx, yy, Z, alpha=0.3, levels=np.arange(n_classes + 1) - 0.5, cmap=cmap, zorder=1
    )

    ax.set(xlim=xlim, ylim=ylim)

Now we can examine what the decision tree classification looks like:

In [ ]:
visualize_classifier(DecisionTreeClassifier(), X, y)

Notice that as the depth increases, we tend to get very strangely shaped classification regions; for example, at a depth of five, there is a tall and skinny purple region between the yellow and blue regions.
It's clear that this is less a result of the true, intrinsic data distribution, and more a result of the particular sampling or noise properties of the data.
That is, this decision tree, even at only five levels deep, is clearly over-fitting our data.

### Decision trees and over-fitting

Such over-fitting turns out to be a general property of decision trees: it is very easy to go too deep in the tree, and thus to fit details of the particular data rather than the overall properties of the distributions they are drawn from.
Another way to see this over-fitting is to look at models trained on different subsets of the data—for example, in this figure we train two different trees, each on half of the original data:

![](https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/05.08-decision-tree-overfitting.png?raw=1)

It is clear that in some places, the two trees produce consistent results (e.g., in the four corners), while in other places, the two trees give very different classifications (e.g., in the regions between any two clusters).
The key observation is that the inconsistencies tend to happen where the classification is less certain, and thus by using information from *both* of these trees, we might come up with a better result!

Just as using information from two trees improves our results, we might expect that using information from many trees would improve our results even further.

## Ensembles of Estimators: Random Forests

This notion—that multiple overfitting estimators can be combined to reduce the effect of this overfitting—is what underlies an ensemble method called *bagging*.
Bagging makes use of an ensemble (a grab bag, perhaps) of parallel estimators, each of which over-fits the data, and averages the results to find a better classification.
An ensemble of randomized decision trees is known as a *random forest*.

This type of bagging classification can be done manually using Scikit-Learn's ``BaggingClassifier`` meta-estimator, as shown here:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

tree = DecisionTreeClassifier()
bag = BaggingClassifier(tree, n_estimators=100, max_samples=0.8, random_state=1)

bag.fit(X, y)
visualize_classifier(bag, X, y)

In this example, we have randomized the data by fitting each estimator with a random subset of 80% of the training points.
In practice, decision trees are more effectively randomized by injecting some stochasticity in how the splits are chosen: this way all the data contributes to the fit each time, but the results of the fit still have the desired randomness.
For example, when determining which feature to split on, the randomized tree might select from among the top several features.
You can read more technical details about these randomization strategies in the [Scikit-Learn documentation](http://scikit-learn.org/stable/modules/ensemble.html#forest) and references within.

In Scikit-Learn, such an optimized ensemble of randomized decision trees is implemented in the ``RandomForestClassifier`` estimator, which takes care of all the randomization automatically.
All you need to do is select a number of estimators, and it will very quickly (in parallel, if desired) fit the ensemble of trees:

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=0)
visualize_classifier(model, X, y);

We see that by averaging over 100 randomly perturbed models, we end up with an overall model that is much closer to our intuition about how the parameter space should be split.

## Random Forest Regression

In the previous section we considered random forests within the context of classification.
Random forests can also be made to work in the case of regression (that is, continuous rather than categorical variables). The estimator to use for this is the ``RandomForestRegressor``, and the syntax is very similar to what we saw earlier.

Consider the following data, drawn from the combination of a fast and slow oscillation:

In [ ]:
rng = np.random.RandomState(42)
x = 10 * rng.rand(200)


def model(x, sigma=0.3):
    fast_oscillation = np.sin(5 * x)
    slow_oscillation = np.sin(0.5 * x)
    noise = sigma * rng.randn(len(x))

    return slow_oscillation + fast_oscillation + noise


y = model(x)
plt.errorbar(x, y, 0.3, fmt="o");

Using the random forest regressor, we can find the best fit curve as follows:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(200)
forest.fit(x[:, None], y)

xfit = np.linspace(0, 10, 1000)
yfit = forest.predict(xfit[:, None])
ytrue = model(xfit, sigma=0)

plt.errorbar(x, y, 0.3, fmt="o", alpha=0.5)
plt.plot(xfit, yfit, "-r")
plt.plot(xfit, ytrue, "-k", alpha=0.5);

Here the true model is shown in the smooth gray curve, while the random forest model is shown by the jagged red curve.
As you can see, the non-parametric random forest model is flexible enough to fit the multi-period data, without us needing to specifying a multi-period model!

## Example: Predicting diamonds price

We use again the example we explored for linear models: we will try and predict the price of diamonds.

In [ ]:
import pandas as pd

diamonds_raw = pd.read_csv(
    "https://raw.githubusercontent.com/tidyverse/ggplot2/master/data-raw/diamonds.csv"
)
diamonds = diamonds_raw.sample(5000, random_state=0)
diamonds.head()

As before, we remove the data with zero dimension.

In [ ]:
diamonds = diamonds[diamonds.x * diamonds.y * diamonds.z != 0]

And we exploit the results of the previous exploratory analysis to understand which variables to keep.

In [ ]:
diamonds_processed = diamonds.drop(columns=["depth", "table", "y", "z"])
diamonds_processed.head()

Then we create the train and the test set.

In [ ]:
from sklearn.model_selection import train_test_split

x = diamonds_processed.drop(columns="price")
y = diamonds_processed.price

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

And finally we create the pipeline for the model. We use the ordinal encoder as all the categorical variables have a natural ordering.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor

column_transformer = make_column_transformer(
    (
        OrdinalEncoder(categories=[["Fair", "Good", "Very Good", "Premium", "Ideal"]]),
        ["cut"],
    ),
    (
        OrdinalEncoder(
            categories=[["I1", "SI2", "SI1", "VS2", "VS1", "VVS2", "VVS1", "IF"]]
        ),
        ["clarity"],
    ),
    (OrdinalEncoder(categories=[["J", "I", "H", "G", "F", "E", "D"]]), ["color"]),
    remainder="passthrough",
)
rf_model = RandomForestRegressor(n_estimators=1000, max_depth=5, random_state=42)

rf_pipeline = make_pipeline(column_transformer, rf_model)

We fit the model to the train set and we assess the metrics on the test set.

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

rf_pipeline.fit(x_train, y_train)
pred = rf_pipeline.predict(x_test)

print(f"R2 Score: {round(r2_score(y_test, pred), 4)}")
print(f"MAE: {round(mean_absolute_error(y_test, pred), 2)}$")

In [ ]:
def plot_gof(y_true, y_pred):
    plt.plot(y_test, pred, ".")
    plt.plot(y_test, y_test, linewidth=3, c="black")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.show()


plot_gof(y_test, pred)

We see that there are no predictions below zero and that the cheaper diamods have better predictions. Maybe a Gradient Boosting model could do better?
We will not try and assess it, but we will try to improve out Random Forest. 

For instance, by trying and optimizing the hyperparameters.

In [ ]:
from sklearn.model_selection import GridSearchCV

rf_model_cv = RandomForestRegressor(random_state=42, n_jobs=-1)

cv_grid = {
    "n_estimators": [200],
    "max_depth": [5, 7, 9],
    "criterion": ["squared_error"],
    "min_samples_split": [2, 4, 8],
}

rf_pipeline = make_pipeline(
    column_transformer, GridSearchCV(rf_model_cv, cv_grid, cv=5, verbose=2)
)

In [ ]:
rf_pipeline.fit(x_train, y_train)
pred = rf_pipeline.predict(x_test)

print(f"R2 Score: {round(r2_score(y_test, pred), 4)}")
print(f"MAE: {round(mean_absolute_error(y_test, pred), 2)}$")

In [ ]:
def plot_gof(y_true, y_pred):
    plt.plot(y_test, pred, ".")
    plt.plot(y_test, y_test, linewidth=3, c="black")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
    plt.show()


plot_gof(y_test, pred)

We have improved a lot! Let's check out the best hyperparameters.

In [ ]:
rf_pipeline[1].best_params_

Now, we hit the limits of the grid for both `max_depth` and `n_estimators`. Possibly, a different value for such hyperparameters could have resulted in better performance. 

Finally, let's have a look to the feature importance.

In [ ]:
forest_importances = pd.Series(
    rf_pipeline[1].best_estimator_.feature_importances_, index=x_train.columns
)
forest_importances.plot.bar();

In [ ]:
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(
    rf_pipeline, x_test, y_test, n_repeats=20, random_state=42
)
pd.Series(perm_importance["importances_mean"], index=x_train.columns).plot.bar();

We see that the two metrics do not coincide. From what we know about the dataset, we are more incline to believe to the permutation importance.